#Задание 1

In [1]:
import os
import json
import ast
import time
import datetime
import numpy as np
import pandas as pd
pd.set_option('max_columns', None)
import matplotlib.pyplot as plt
import seaborn
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

In [0]:
from scipy import stats
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, cross_val_score, GridSearchCV, RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn import linear_model
from sklearn.neighbors import NearestNeighbors

In [0]:
import warnings
warnings.filterwarnings('ignore')

Датасет

In [4]:
from sklearn.datasets import fetch_20newsgroups
categories = ['sci.crypt', 'sci.electronics', 'sci.med', 'sci.space',] 
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

In [0]:
from sklearn.metrics import *
twenty_test = fetch_20newsgroups(subset='test', categories=categories, 
                                 shuffle=True, random_state=0)

Векторизация

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

In [0]:
X = count_vect.fit_transform(twenty_train.data)
y = twenty_train.target

In [0]:
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, cross_val_score, GridSearchCV, RepeatedStratifiedKFold

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.70, random_state=42)

In [0]:
stratified_folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

Логистическая регрессия 

In [0]:
from sklearn.metrics import f1_score

In [0]:
def cv_train_model(X, y, X_test, y_test, folds=stratified_folds, plot_feature_importance=False, model=None):
    oof = np.zeros(len(X))
    prediction = np.zeros(len(X_test))
    valid_scores = []
    feature_importance = pd.DataFrame()
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X, y)):
        X_train, X_valid = X[train_index], X[valid_index]
        y_train, y_valid = y[train_index], y[valid_index]
          

        model.fit(X_train, y_train)      
        y_pred_test = model.predict_proba(X_test)[:, 1]
        y_pred_valid = model.predict_proba(X_valid)[:, 1]
        valid_score = f1_score(y_valid, np.round(y_pred_valid), average= 'macro')
        
        oof[valid_index] = y_pred_valid.reshape(-1,)
        valid_scores.append(valid_score)
        prediction += y_pred_test  
        
    prediction /= (fold_n + 1)
    test_score = f1_score(y_test, np.round(prediction), average= 'macro')
    print('Cross Validation mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(valid_scores), np.std(valid_scores)))
    print("Score on test data: {0:.4f}".format(test_score))
    

    return oof, prediction, valid_scores

In [0]:
%%time

#np.arange(0,2,.5)
#np.linspace(0,2,10)

lr = linear_model.LogisticRegression()

parameter_grid = {'class_weight' : ['balanced', None],
                  'penalty' : ['l2', 'l1'],
                  'solver' : ['liblinear', 'saga'],
                  'C' : [0.001, 0.01, 0.08, 0.1, 0.15, 1.0, 10.0, 100.0],
                  'max_iter': [2,10,50,100,1000]
                 }

grid_search = GridSearchCV(lr, param_grid=parameter_grid, cv=stratified_folds, scoring = 'f1_macro')
grid_search.fit(X_train, y_train)
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

Best score: 0.8958928395548114
Best parameters: {'C': 10.0, 'class_weight': None, 'max_iter': 50, 'penalty': 'l2', 'solver': 'liblinear'}
CPU times: user 2h 15min 17s, sys: 1min 15s, total: 2h 16min 33s
Wall time: 2h 15min 48s


In [14]:
lr = linear_model.LogisticRegression(C = 10.0, class_weight = None, max_iter = 50, penalty = 'l2', solver = 'liblinear', multi_class= 'ovr')
oof_lr, prediction_lr, scores = cv_train_model(X_train.toarray(), y_train, X_test.toarray(), y_test,  model=lr, folds = stratified_folds)

Cross Validation mean score: 0.3289, std: 0.0060.
Score on test data: 0.3411


Деревья решений

In [0]:
from sklearn.tree import DecisionTreeClassifier

In [0]:
%%time

#np.arange(0,2,.5)
#np.linspace(0,2,10)

dtc = DecisionTreeClassifier()

parameter_grid = {'criterion' : ['gini', 'entropy'], 'max_depth' : [4,6,8,12,24,32]}

grid_search = GridSearchCV(dtc, param_grid=parameter_grid, cv=stratified_folds, scoring = 'f1_macro')
grid_search.fit(X_train, y_train)
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

Best score: 0.7042198539436261
Best parameters: {'criterion': 'gini', 'max_depth': 24}
CPU times: user 8.14 s, sys: 12.6 ms, total: 8.16 s
Wall time: 8.22 s


In [55]:
dtc = DecisionTreeClassifier(criterion = 'gini', max_depth = 24)
oof_dtc, prediction_dtc, scores = cv_train_model(X_train.toarray(), y_train, X_test.toarray(), y_test,  model=dtc, folds = stratified_folds)

Cross Validation mean score: 0.2550, std: 0.0215.
Score on test data: 0.2974


SGD
 

In [0]:
from sklearn.linear_model import SGDClassifier

In [0]:
%%time

#np.arange(0,2,.5)
#np.linspace(0,2,10)

sgdc = SGDClassifier()

parameter_grid = {'penalty' : ['l2', 'l1'], 'alpha' : [0.01,0.2,5,10], 'shuffle' : [True, False], 'n_iter_no_change' : [2, 5, 10]}

grid_search = GridSearchCV(sgdc, param_grid=parameter_grid, cv=stratified_folds, scoring = 'f1_macro') 
grid_search.fit(X_train, y_train)
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

Best score: 0.8958114240151479
Best parameters: {'alpha': 0.2, 'n_iter_no_change': 5, 'penalty': 'l2', 'shuffle': True}
CPU times: user 1min 29s, sys: 1min 12s, total: 2min 41s
Wall time: 1min 17s


In [56]:
sgdc = SGDClassifier(alpha = 0.2, n_iter_no_change = 5, penalty = 'l2', shuffle = True, loss = 'log')
oof_sgdc, prediction_sgdc, scores = cv_train_model(X_train.toarray(), y_train, X_test.toarray(), y_test,  model=sgdc, folds = stratified_folds)

Cross Validation mean score: 0.2554, std: 0.0210.
Score on test data: 0.2683


#Результаты
Качество классификаторов оставляет желать лучшего. Лучший результат на тесте показала логистическая регрессия (0.34). 

#Задание 2

In [0]:
!pip install eli5

In [0]:
import eli5

In [0]:
index_to_word = {v:k for k,v in count_vect.vocabulary_.items()}

In [0]:
def analyze_features(model, n):

  top_dtc = []
  if model == dtc:
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]
    for index in indices:
      if n:
        print(index_to_word[index])
        n -= 1
      else:
        continue
  else:
    feats = eli5.formatters.as_dataframe.explain_weights_df(model)
    feats0 = feats[feats['target'] == 0]
    feats1 = feats[feats['target'] == 1]
    feats2 = feats[feats['target'] == 2]
    feats3 = feats[feats['target'] == 3]
    top_features_0 = [int(i[1:]) for i in feats0.feature if 'BIAS' not in i]
    top_features_1 = [int(i[1:]) for i in feats1.feature if 'BIAS' not in i]
    top_features_2 = [int(i[1:]) for i in feats2.feature if 'BIAS' not in i]
    top_features_3 = [int(i[1:]) for i in feats3.feature if 'BIAS' not in i]
    top0 = []
    top1 = []
    top2 = []
    top3 = []

    while len(top0) < n or len(top1) < n or len(top2) < n or len(top3) < n:
      for index in top_features_0:
        word = index_to_word[index]
        if len(top0) < n:
          top0.append(word)
        else:
          continue
      for index in top_features_1:
        word = index_to_word[index]
        if len(top1) < n:
          top1.append(word)
        else:
          continue
      for index in top_features_2:
        word = index_to_word[index]
        if len(top2) < n:
          top2.append(word)
        else:
          continue
      for index in top_features_3:
        word = index_to_word[index]
        if len(top3) < n:
          top3.append(word)
        else:
          continue

    print('Top sci.crypt features:', [word for word in top0])
    print('Top sci.electronics features:', [word for word in top1])
    print('Top sci.med features:', [word for word in top2])
    print('Top sci.space features:', [word for word in top3])




In [58]:
analyze_features(sgdc, 5)

Top sci.crypt features: ['key', 'encryption', 'clipper', 'chip', 'privacy']
Top sci.electronics features: ['wire', 'use', 'wiring', 'with', 'power']
Top sci.med features: ['msg', 'is', 'pitt', 'doctor', 'medical']
Top sci.space features: ['space', 'launch', 'henry', 'nasa', 'orbit']


In [59]:
analyze_features(lr, 5)

Top sci.crypt features: ['clipper', 'key', 'encryption', 'chip', 'crypto']
Top sci.electronics features: ['rf', 'hp', 'au', 'mike', '10']
Top sci.med features: ['msg', 'doctor', 'health', 'treatment', 'erythromycin']
Top sci.space features: ['space', 'orbit', 'henry', 'nasa', 'funding']


In [60]:
analyze_features(dtc, 5)

clipper
space
key
gordon
foods


#Задание 3

In [0]:
from string import punctuation

In [0]:
punct = punctuation+'«»—…“”*№–'

def normalize_punct(text):
    words = [word.strip(punct) for word in text.split()]
    return ' '.join(words)

In [0]:
twenty_train_norm = [] 
for text in twenty_train.data:
  text_norm = normalize_punct(text)
  twenty_train_norm.append(text_norm)


In [0]:
count_vect2 = CountVectorizer(stop_words = 'english', max_df = 0.3, min_df = 5, ngram_range=(1,3)) 

In [0]:
X_new = count_vect2.fit_transform(twenty_train_norm)
y_new = twenty_train.target

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.70, random_state=42)

In [265]:
lr = linear_model.LogisticRegression(C = 10.0, class_weight = None, max_iter = 50, penalty = 'l2', solver = 'liblinear', multi_class= 'ovr')
oof_lr, prediction_lr, scores = cv_train_model(X_train.toarray(), y_train, X_test.toarray(), y_test,  model=lr, folds = stratified_folds)

#1 вариант
#Cross Validation mean score: 0.3289, std: 0.0060.
#Score on test data: 0.3411

Cross Validation mean score: 0.3347, std: 0.0056.
Score on test data: 0.3482


In [266]:
dtc = DecisionTreeClassifier(criterion = 'gini', max_depth = 24)
oof_dtc, prediction_dtc, scores = cv_train_model(X_train.toarray(), y_train, X_test.toarray(), y_test,  model=dtc, folds = stratified_folds)

#1 вариант
#Cross Validation mean score: 0.2550, std: 0.0215.
#Score on test data: 0.2974

Cross Validation mean score: 0.2477, std: 0.0403.
Score on test data: 0.3013


In [267]:
sgdc = SGDClassifier(alpha = 0.2, n_iter_no_change = 5, penalty = 'l2', shuffle = True, loss = 'log')
oof_sgdc, prediction_sgdc, scores = cv_train_model(X_train.toarray(), y_train, X_test.toarray(), y_test,  model=sgdc, folds = stratified_folds)

#1 вариант
#Cross Validation mean score: 0.2554, std: 0.0210.
#Score on test data: 0.2683

Cross Validation mean score: 0.2563, std: 0.0284.
Score on test data: 0.2707


#Результаты
Качество удалось повысить, однако совсем незначительно. Сначала я убрал стоп-слова, т. к. среди самых весомых фичей категории sci.med у SGDClassifier было 'is'. Затем удалил пунктуацию и подкрутил параметры max_df и min_df. Можно было бы найти показатели и получше, но в таком случае зачастую повышается качество одного классификатора, а качество других падает. Такие параметры дают пусть и небольшой прирост в качестве, зато прирост этот получают все классификаторы. 